In [49]:
import matplotlib.pyplot as plt
import random

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

import torch
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

In [50]:
df = pd.read_csv("../../data/image_classification.csv")

x = df.iloc[:, :-1]
y = pd.get_dummies(df.iloc[:, -1]).astype(int)

x_ssc = StandardScaler()
x_scaled = x_ssc.fit_transform(x)

x_tensor = torch.tensor(x_scaled,dtype=torch.float32)
y_tensor = torch.tensor(y.values,dtype=torch.float32)

x_train, x_test, y_train, y_test = train_test_split(x_tensor, y_tensor, test_size=0.2, shuffle=True)

In [51]:
# USE_mps = torch.backends.mps.is_available() # GPU를 사용가능하면 True, 아니라면 False를 리턴

# device = torch.device("mps" if USE_mps else "cpu") # GPU 사용 가능하면 사용하고 아니면 CPU 사용
# print("다음 기기로 학습합니다:", device)

In [52]:
# DataLoader 만들기
# 1. tensor dataset 생성
dataset = TensorDataset(x_train, y_train)

# 2. DataLoader 에 적절한 batch_size와 함께 입력
dataloader = DataLoader(
    dataset=dataset,
    batch_size=32,
    shuffle=True,
    drop_last=True
)

In [57]:
# model
class Model(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.linear_1 = nn.Linear(64, 32)
        self.linear_2 = nn.Linear(32, 16)
        self.linear_3 = nn.Linear(16, 10)
    
    def forward(self, x):
        x = self.linear_1(x)
        x = self.linear_2(x)
        x = self.linear_3(x)
        return x

In [58]:
model = Model()

In [59]:
# 반복 설정
epochs = 15
# optimizer 설정
optimizer = optim.SGD(model.parameters(),lr=0.1)
# loss function 설정
loss_fn = nn.CrossEntropyLoss()

In [60]:
for _ in range(epochs):
    for X,Y in dataloader:
        optimizer.zero_grad()
        y_pred = model(x_train)
        loss = loss_fn(y_pred,y_train)
        
        # 미분
        loss.backward()
        # parameter 미분값에 대해 수정
        optimizer.step()
    print(loss.item())

0.65453040599823
0.3157190680503845
0.19617971777915955
0.14538446068763733
0.11710651963949203
0.09824054688215256
0.08432058990001678
0.07349277287721634
0.06482931971549988
0.057754404842853546
0.0518684946000576
0.04688979312777519
0.042618364095687866
0.03891037404537201
0.03566073626279831


In [65]:
accuracy_score(model(x_test).argmax(axis=1),y_test.argmax(axis=1))

0.9694444444444444